# Pytorch - Datasets

Vamos a ver ahora caracteristicas de pytorch para gestionar datasets

In [1]:
import torch

## Iterando tensores

A continuación tenemos una implementación en la que iteramos por los datos de manera explícita para entrenar nuestra red.

In [2]:
torch.cuda.is_available()

False

In [9]:
from sklearn.datasets import fetch_openml
import numpy as np

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)

X_t = torch.from_numpy(X_train).float().cpu()
Y_t = torch.from_numpy(y_train).long().cpu()

<ipython-input-9-798b0ddf8d23>:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)


In [10]:
from sklearn.metrics import accuracy_score

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

In [11]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cpu())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.74829
Epoch 20/100 Loss 1.51558
Epoch 30/100 Loss 1.22416
Epoch 40/100 Loss 1.04271
Epoch 50/100 Loss 0.91677
Epoch 60/100 Loss 0.82456
Epoch 70/100 Loss 0.75561
Epoch 80/100 Loss 0.69651
Epoch 90/100 Loss 0.64907
Epoch 100/100 Loss 0.61085


0.9231

## Iterando por *Batches*

En la implementación anterior estamos optimizando nuestro modelo con el algoritmo de `batch gradient descent`, en el que utilizamos todos nuestros datos en cada paso de optimización. Sin embargo, un algoritmo que puede converger más rápido (y única opción si nuestro dataset es tan grande que no cabe en memoria) es el de `mini-batch gradient descent`.

In [5]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(X_t) // batch_size
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches
    for b in range(batches):
        x_b = X_t[b*batch_size:(b+1)*batch_size]
        y_b = Y_t[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cpu())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.31910
Epoch 2/10 Loss 0.22426
Epoch 3/10 Loss 0.17940
Epoch 4/10 Loss 0.15200
Epoch 5/10 Loss 0.13268
Epoch 6/10 Loss 0.11809
Epoch 7/10 Loss 0.10661
Epoch 8/10 Loss 0.09718
Epoch 9/10 Loss 0.08923
Epoch 10/10 Loss 0.08240


0.9752

Si bien esta implementación es correcta y funcional, dependiendo de nuestros datos puede llegar a complicarse mucho (por ejemplo, si necesitamos cargar muchas imágenes a las cuales queremos aplicar transformaciones, juntar en batches, etc...). Además, es común reutilizar la lógica para cargar nuestros datos no sólo para entrenar la red, si no para generar predicciones. Este hecho motiva el uso de las clases especiales que `Pytorch` nos ofrece para ello.

## La clase *Dataset*

La primera clase que tenemos que conocer es la clase `Dataset`. Esta clase hereda de la clase madre `torch.utils.data.Dataset` y tenemos que definir, como mínimo, tres funciones: 

- `__init__`: el constructor
- `__len__`: devuelve el número de muestras en el dataset
- `__getitem__`: devuelve una muestra en concreto del dataset

Una vez definida la clase, ésta puede usarse como si de cualquier iterador se tratase.

In [ ]:
# clase Dataset, hereda de la clase `torch.utils.data.Dataset`
class Dataset(torch.utils.data.Dataset):
    # constructor
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float().cpu()
        self.Y = torch.from_numpy(Y).long().cpu()
    # devolvemos el número de datos en el dataset
    def __len__(self):
        return len(self.X)
    # devolvemos el elemento `ix` del dataset
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]

Una vez definida la clase, podemos instanciar un objeto que podemos usar para iterar por nuestros datos.

In [ ]:
dataset = Dataset(X_train, y_train)

len(dataset)

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(dataset) // batch_size
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataset
    for b in range(batches):
        x_b, y_b = dataset[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cpu())
accuracy_score(y_test, y_pred.cpu().numpy())

Podemos iterar directamente sobre el objeto `dataset` de la misma manera que hacíamos anteriormente, sin embargo `Pytorch` no ofrece otro objeto que nos facilita las cosas a la hora de iterar por batches.

## La clase *DataLoader*

La clase `DataLoader` recibe un `Dataset` e implementa la lógica para iterar nuestros datos en batches.

In [11]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

In [12]:
x, y = next(iter(dataloader))

x.shape, y.shape

(torch.Size([100, 784]), torch.Size([100]))

También permite mezclar los datos al principio de cada epoch con el parámetro `shuffle`, de manera automática carga nuestros datos de manera optimizada utilizando varios *cores* de nuestra CPU si es posible, etc.

In [14]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
log_each = 1
l = []
model.train()
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataloader
    for x_b, y_b in dataloader:
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cpu())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.29677
Epoch 2/10 Loss 0.20535
Epoch 3/10 Loss 0.16461
Epoch 4/10 Loss 0.14059
Epoch 5/10 Loss 0.12344
Epoch 6/10 Loss 0.11093
Epoch 7/10 Loss 0.10083
Epoch 8/10 Loss 0.09241
Epoch 9/10 Loss 0.08542
Epoch 10/10 Loss 0.07934


0.9767

También permite definir nuestra propia lógica para crear los batches, algo que puede ser útil en ciertas ocasiones.

In [15]:
def collate_fn(batch):
    return torch.stack([x for x, y in batch]), torch.stack([y for x, y in batch]), torch.stack([2.*x for x, y in batch])

In [16]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)

In [17]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cpu")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
log_each = 1
l = []
model.train()
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataloader
    # no usamos x2_b, sólo es para ver un ejemplo
    for x_b, y_b, x2_b in dataloader:
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cpu())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.29013
Epoch 2/10 Loss 0.20268
Epoch 3/10 Loss 0.16396
Epoch 4/10 Loss 0.14023
Epoch 5/10 Loss 0.12347
Epoch 6/10 Loss 0.11073
Epoch 7/10 Loss 0.10035
Epoch 8/10 Loss 0.09189
Epoch 9/10 Loss 0.08471
Epoch 10/10 Loss 0.07858


0.9772

## Resumen

Hemos visto diferentes maneras en las que podemos iterar por nuestros datos para entrenar un modelo en `Pytorch`. Si nuestro dataset es sencillo y podemos representarlo como un simple `array` de `NumPy` podemos iterar directamente el `array`, transformándolo previamente en un `tensor`. Sin embargo, cuando nuestro dataset sea más grande y no quepa en memoria o necesite cierto pre-proceso o transformaciones, es muy conveniente utilizar las clases que `Pytorch` nos ofrece para ello. Estas clases son, principalmente, el `Dataset` y el `DataLoader`, las cuales nos van a permitir iterar por nuestros datos de manera eficiente y generar *batches* de forma sencilla (además de otras funcionalidades como mezclar los datos al principio de cada epoch, cargar datos en paralelo, etc).